<h1><center>CMPE 462 - Project 2<br>Implementing an SVM Classifier<br>Due: May 18, 2020, 23:59</center></h1>

* **Student ID1:** 2014400051
* **Student ID2:** 2018400291
* **Student ID3:** 2019705072

## Overview

In this project, you are going to implement SVM. For this purpose, a data set (data.mat) is given to you. You can load the mat dataset into Python using the function `loadmat` in `Scipy.io`. When you load the data, you will obtain a dictionary object, where `X` stores the data matrix and `Y` stores the labels. You can use the first 150 samples for training and the rest for testing. In this project, you will use the software package [`LIBSVM`](http://www.csie.ntu.edu.tw/~cjlin/libsvm/) to implement SVM. Note that `LIBSVM` has a [`Python interface`](https://github.com/cjlin1/libsvm/tree/master/python), so you can call the SVM functions in Python. 

In [1]:
# Installing required libraries before we start using them
!pip3 install libsvm 
!pip3 install PTable

In [2]:
import scipy.io as spio  #Import required libraries
from libsvm.svmutil import *
import numpy as np
from prettytable import PrettyTable

dic = spio.loadmat("data.mat")

train_matrix, train_labels = dic['X'][:150], dic['Y'][:150] # Train set and labels
test_matrix, test_labels = dic['X'][150:], dic['Y'][150:] # Test set and labels

train_labels = train_labels.reshape((train_labels.shape[0],)) #Convert datasets to 1D numpy array shape
test_labels = test_labels.reshape((test_labels.shape[0],))

## Task 1 - 30 pts

Train a hard margin linear SVM and report both train and test classification accuracy.

In [3]:
# This function takes a parameter for svm_train method of libsvm package
# trains a model using the train set of data.mat
# uses this model to classify samples
# Calculates both train accuracy and test accuracy

def svm_function(params):
    global train_labels, train_matrix, test_labels, test_matrix # data is obtained from global test and train data.
    m = svm_train(train_labels, train_matrix, params) # libsvm function to train model with train set
    
     # Predict accuracy of Training set
    result = [] # result is will be a tuple of train accuracy and test accuracy
    p_label, p_acc, p_val = svm_predict(train_labels, train_matrix, m, '-q') # Model predictions and accuracy are returned from the function.
    result.append(str(round(p_acc[0], 4))) # train set accuracy is rounded to 4 decimal points and returned as string
    
    # Predict accuracy of Test set
    p_label, p_acc, p_val = svm_predict(test_labels, test_matrix, m, '-q') # -q is abbreviation for quite mode, prevents function to output results
    result.append(str(round(p_acc[0], 4))) # test set accuracy is rounded to 4 decimal points and returned as string
    return result

In [4]:
res = svm_function('-t 0 -c 100000000') # Hard margin SVM, c-value is selected very large number, -t 0 means linear kernel
print('Hard Margin SVM')
print('Train classification accuracy: ' + res[0])
print('Test classification accuracy: ' + res[1])

Hard Margin SVM
Train classification accuracy: 74.6667
Test classification accuracy: 77.5


## Task 2 - 40 pts

Train soft margin SVM for different values of the parameter $C$, and with different kernel functions. Systematically report your results. For instance, report the performances of different kernels for a fixed $C$, then report the performance for different $C$ values for a fixed kernel, and so on.

## Fixed Kernel, Different C-values

In [5]:
# Radial basis function: exp(-1*|u-v|^2)
c_v = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000] # Different c values to train models
results = []

# -t 2 means RBF Kernel
for c in c_v: # for every c-value we create new parameters to be sent to our svm_function
    results.append(svm_function(f'-t 2 -c {c}')) 

# Pretty table helps us visualise results with changing parameters in a nice way
# You can add more c-values to the list c_v and run the cell again. 
table = PrettyTable() 
table.title = 'Kernel -> Radial basis function : exp(-1*|u-v|^2)'
table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results]) # First element of tuple is training accuraccy
table.add_row(["Test Accuracy"]+[res[1] for res in results]) # Second element of tuple is test accuracy
print(table)

+------------------------------------------------------------------------------------------------+
|                       Kernel -> Radial basis function : exp(-1*|u-v|^2)                        |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
|    C Value     |  0.001  |   0.01  |   0.1   |    1    |    10   |   100   |   1000  |  10000  |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
| Train Accuracy | 53.3333 | 53.3333 | 83.3333 | 86.6667 | 95.3333 | 99.3333 |  100.0  |  100.0  |
| Test Accuracy  | 58.3333 | 58.3333 | 84.1667 | 84.1667 |   77.5  | 78.3333 | 76.6667 | 76.6667 |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+


In [6]:
# Kernel: Polynomial: (u'*v)^2

c_v = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
results = []

# -t 1 means Polynomial Kernel
# -d 2 means degree

for c in c_v: # for every c-value we create new parameters to be sent to our svm_function
    results.append(svm_function(f'-t 1 -d 2 -c {c}'))

table = PrettyTable()
table.title ='Kernel -> Polynomial: (u\'*v)^2'

table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+------------------------------------------------------------------------------------------------+
|                                 Kernel -> Polynomial: (u'*v)^2                                 |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
|    C Value     |  0.001  |   0.01  |   0.1   |    1    |    10   |   100   |   1000  |  10000  |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
| Train Accuracy | 53.3333 | 53.3333 |   56.0  | 85.3333 | 88.6667 | 97.3333 | 99.3333 |  100.0  |
| Test Accuracy  | 58.3333 | 58.3333 | 59.1667 | 78.3333 | 79.1667 | 76.6667 | 73.3333 | 69.1667 |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+


In [7]:
# Kernel: Sigmoid tanh(u'*v)

c_v = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
results = []

# -t 3 means Sigmoid Kernel
for c in c_v:
    results.append(svm_function(f'-t 3 -c {c}'))

table = PrettyTable()
table.title ='Kernel -> Sigmoid: tanh(u\'*v)'

table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+---------------------------------------------------------------------------------------------+
|                                Kernel -> Sigmoid: tanh(u'*v)                                |
+----------------+---------+---------+---------+---------+------+---------+---------+---------+
|    C Value     |  0.001  |   0.01  |   0.1   |    1    |  10  |   100   |   1000  |  10000  |
+----------------+---------+---------+---------+---------+------+---------+---------+---------+
| Train Accuracy | 53.3333 | 53.3333 |   82.0  | 82.6667 | 78.0 | 76.6667 | 75.3333 | 75.3333 |
| Test Accuracy  | 58.3333 | 58.3333 | 84.1667 | 84.1667 | 80.0 |   72.5  | 74.1667 | 74.1667 |
+----------------+---------+---------+---------+---------+------+---------+---------+---------+


In [8]:
# Kernel: Linear: u'*v

c_v = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
results = []

# -t 0 means Linear Kernel
for c in c_v:
    results.append(svm_function(f'-t 0 -c {c}'))

table = PrettyTable()
table.title ='Kernel -> Linear: u\'*v'

table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+------------------------------------------------------------------------------------------------+
|                                     Kernel -> Linear: u'*v                                     |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
|    C Value     |  0.001  |   0.01  |   0.1   |    1    |    10   |   100   |   1000  |  10000  |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
| Train Accuracy | 53.3333 | 82.6667 |   86.0  | 86.6667 | 88.6667 | 88.6667 |   90.0  |   90.0  |
| Test Accuracy  | 58.3333 | 84.1667 | 83.3333 |   85.0  | 81.6667 | 81.6667 | 81.6667 | 81.6667 |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+


In [9]:
# Kernel: Polynomial: (u'*v)^3

c_v = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
results = []

# -t 1 means Polynomial Kernel
# -d 3 means degree 3
for c in c_v:
    results.append(svm_function(f'-t 1 -d 3 -c {c}'))

table = PrettyTable()
table.title ='Kernel -> Polynomial: (u\'*v)^3'

table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+---------------------------------------------------------------------------------------------+
|                                Kernel -> Polynomial: (u'*v)^3                               |
+----------------+---------+---------+---------+------+---------+---------+---------+---------+
|    C Value     |  0.001  |   0.01  |   0.1   |  1   |    10   |   100   |   1000  |  10000  |
+----------------+---------+---------+---------+------+---------+---------+---------+---------+
| Train Accuracy | 53.3333 | 53.3333 | 53.3333 | 86.0 |   94.0  | 98.6667 |  100.0  |  100.0  |
| Test Accuracy  | 58.3333 | 58.3333 | 58.3333 | 82.5 | 80.8333 |   75.0  | 75.8333 | 75.8333 |
+----------------+---------+---------+---------+------+---------+---------+---------+---------+


## Fixed C, Different Kernels

In [18]:
kernels = ['-t 1 -d 3', '-t 1 -d 2', '-t 0', '-t 3', '-t 2'] 
kernel_names = ['polynomial(U\'V)^3', 'polynomial(U\'V)^2', 'linear', 'sigmoid', 'RBF']

# kernels and kernel_names lists are complementary. Elements of both lists with same indices are pairs.
# You can add new kernel parameters to 'kernels' and add its definiton to 'kernel_names' and run the cell again
results = []

# For each kernel parameter, we create new parameters and send to our svm_function for fixed C = 0.01.
for kernel in kernels:
    results.append(svm_function(f'{kernel} -c 0.01'))

table = PrettyTable()
table.title ='Fixed C = 0.01'

table.field_names = ["Kernel"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                    Fixed C = 0.01                                    |
+----------------+-------------------+-------------------+---------+---------+---------+
|     Kernel     | polynomial(U'V)^3 | polynomial(U'V)^2 |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |      53.3333      |      53.3333      | 82.6667 | 53.3333 | 53.3333 |
| Test Accuracy  |      58.3333      |      58.3333      | 84.1667 | 58.3333 | 58.3333 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [19]:
kernels = ['-t 1 -d 3', '-t 1 -d 2', '-t 0', '-t 3', '-t 2'] 
kernel_names = ['polynomial(U\'V)^3', 'polynomial(U\'V)^2', 'linear', 'sigmoid', 'RBF']
results = []

# kernels and kernel_names lists are complementary. Elements of both lists with same indices are pairs.
# You can add new kernel parameters to 'kernels' and add its definiton to 'kernel_names' and run the cell again
for kernel in kernels:
    results.append(svm_function(f'{kernel} -c 0.1'))

# For each kernel parameter, we create new parameters and send to our svm_function for fixed C = 0.1.
table = PrettyTable()
table.title ='Fixed C = 0.1'

table.field_names = ["Kernel"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                    Fixed C = 0.1                                     |
+----------------+-------------------+-------------------+---------+---------+---------+
|     Kernel     | polynomial(U'V)^3 | polynomial(U'V)^2 |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |      53.3333      |        56.0       |   86.0  |   82.0  | 83.3333 |
| Test Accuracy  |      58.3333      |      59.1667      | 83.3333 | 84.1667 | 84.1667 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [20]:
kernels = ['-t 1 -d 3', '-t 1 -d 2', '-t 0', '-t 3', '-t 2'] 
kernel_names = ['polynomial(U\'V)^3', 'polynomial(U\'V)^2', 'linear', 'sigmoid', 'RBF']
results = []

# kernels and kernel_names lists are complementary. Elements of both lists with same indices are pairs.
# You can add new kernel parameters to 'kernels' and add its definiton to 'kernel_names' and run the cell again
for kernel in kernels:
    results.append(svm_function(f'{kernel} -c 1'))

# For each kernel parameter, we create new parameters and send to our svm_function for fixed C = 1.
table = PrettyTable()
table.title ='Fixed C = 1'

table.field_names = ["Kernel"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                     Fixed C = 1                                      |
+----------------+-------------------+-------------------+---------+---------+---------+
|     Kernel     | polynomial(U'V)^3 | polynomial(U'V)^2 |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |        86.0       |      85.3333      | 86.6667 | 82.6667 | 86.6667 |
| Test Accuracy  |        82.5       |      78.3333      |   85.0  | 84.1667 | 84.1667 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [21]:
kernels = ['-t 1 -d 3', '-t 1 -d 2', '-t 0', '-t 3', '-t 2'] 
kernel_names = ['polynomial(U\'V)^3', 'polynomial(U\'V)^2', 'linear', 'sigmoid', 'RBF']
results = []

# kernels and kernel_names lists are complementary. Elements of both lists with same indices are pairs.
# You can add new kernel parameters to 'kernels' and add its definiton to 'kernel_names' and run the cell again
for kernel in kernels:
    results.append(svm_function(f'{kernel} -c 10'))

# For each kernel parameter, we create new parameters and send to our svm_function for fixed C = 10
table = PrettyTable()
table.title ='Fixed C = 10'

table.field_names = ["Kernel"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                     Fixed C = 10                                     |
+----------------+-------------------+-------------------+---------+---------+---------+
|     Kernel     | polynomial(U'V)^3 | polynomial(U'V)^2 |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |        94.0       |      88.6667      | 88.6667 |   78.0  | 95.3333 |
| Test Accuracy  |      80.8333      |      79.1667      | 81.6667 |   80.0  |   77.5  |
+----------------+-------------------+-------------------+---------+---------+---------+


In [22]:
kernels = ['-t 1 -d 3', '-t 1 -d 2', '-t 0', '-t 3', '-t 2'] 
kernel_names = ['polynomial(U\'V)^3', 'polynomial(U\'V)^2', 'linear', 'sigmoid', 'RBF']
results = []

# kernels and kernel_names lists are complementary. Elements of both lists with same indices are pairs.
# You can add new kernel parameters to 'kernels' and add its definiton to 'kernel_names' and run the cell again
for kernel in kernels:
    results.append(svm_function(f'{kernel} -c 100'))

table = PrettyTable()
table.title ='Fixed C = 100'

# For each kernel parameter, we create new parameters and send to our svm_function for fixed C = 100
table.field_names = ["Kernel"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                    Fixed C = 100                                     |
+----------------+-------------------+-------------------+---------+---------+---------+
|     Kernel     | polynomial(U'V)^3 | polynomial(U'V)^2 |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |      98.6667      |      97.3333      | 88.6667 | 76.6667 | 99.3333 |
| Test Accuracy  |        75.0       |      76.6667      | 81.6667 |   72.5  | 78.3333 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [23]:
kernels = ['-t 1 -d 3', '-t 1 -d 2', '-t 0', '-t 3', '-t 2'] 
kernel_names = ['polynomial(U\'V)^3', 'polynomial(U\'V)^2', 'linear', 'sigmoid', 'RBF']
results = []

# kernels and kernel_names lists are complementary. Elements of both lists with same indices are pairs.
# You can add new kernel parameters to 'kernels' and add its definiton to 'kernel_names' and run the cell again
for kernel in kernels:
    results.append(svm_function(f'{kernel} -c 1000'))

# For each kernel parameter, we create new parameters and send to our svm_function for fixed C = 1000
table = PrettyTable()
table.title ='Fixed C = 1000'

table.field_names = ["Kernel"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                    Fixed C = 1000                                    |
+----------------+-------------------+-------------------+---------+---------+---------+
|     Kernel     | polynomial(U'V)^3 | polynomial(U'V)^2 |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |       100.0       |      99.3333      |   90.0  | 75.3333 |  100.0  |
| Test Accuracy  |      75.8333      |      73.3333      | 81.6667 | 74.1667 | 76.6667 |
+----------------+-------------------+-------------------+---------+---------+---------+


## Task 3 - 15 pts

Please report how the number of support vectors changes as the value of $C$ increases (while all other parameters remain the same). Discuss whether your observations match the theory.

In [24]:
# Trains a model with train set using parameters and returns the number of support vectors of the model
def get_number_of_sv(params):
    global train_labels, train_matrix
    m = svm_train(train_labels, train_matrix, params)
    return m.get_nr_sv() # get_nr_sv means number of support vectors, utility function of libsvm

In [25]:
c_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
num_sv = [] # stores number of support vectors for each model trained with certain parameter c

# For each c value, call our get_number_of_sv function and store result in num_sv
for c in c_values:
    num_sv.append(get_number_of_sv(f'-t 0 -c {c}'))
    
table = PrettyTable()

table.title ='Kernel -> Linear: u\'*v' # Model is trained with Linear kernel

table.field_names = ["C Value"]+[str(c) for c in c_values]
table.add_row(["Number of Support Vectors"]+num_sv)
print(table)

+-------------------------------------------------------------------------------+
|                             Kernel -> Linear: u'*v                            |
+---------------------------+-------+------+-----+----+----+-----+------+-------+
|          C Value          | 0.001 | 0.01 | 0.1 | 1  | 10 | 100 | 1000 | 10000 |
+---------------------------+-------+------+-----+----+----+-----+------+-------+
| Number of Support Vectors |  140  | 118  |  74 | 58 | 51 |  50 |  49  |   49  |
+---------------------------+-------+------+-----+----+----+-----+------+-------+


## Discussion

* C is penalty term on the slack variables, measures the degree to which the margin constraints are violated. If we increase C-value, a greater penalty is applied on violators and less data points are tolerated. So, SVM margin will be narrower. Thus, less data points are support vectors. Theory says When we increase value of C, number of support vectors decreases. Our observations match the theory. 

## Task 4 - 15 pts

Please investigate the changes in the hyperplane when you remove one of the support vectors, vs., one data point that is not a support vector.

In [26]:
#takes data matrix and trained model, returns weight, number of support vectors and norm of weight
def get_weight(data,m):
    
    #declaring coefficent matrix and matrix of support vectors
    SV_coef = np.zeros(shape=(m.get_nr_sv()))
    SVs = np.zeros(shape=(m.get_nr_sv(),data.shape[1]))

    #filling coefficent matrix and matrix of support vectors with proper values
    for index in range(0,m.get_nr_sv()):
        SV_coef[index] = m.get_sv_coef()[index][0]
        SVs[index] = data[m.get_sv_indices()[index]-1]
        
    #calculating weight vector via dot product of coefficent matrix and matrix of support vectors 
    weight = np.dot(SV_coef.transpose(),SVs)

    return weight, m.get_nr_sv() ,np.linalg.norm(weight)

In [27]:
#deletes a support vector with given index, returns weight, number of support vectors and norm of weight
def delete_sv(data,labels,m,param,index):
    
    #deletes proper row of data matrix
    new_set = np.delete(data,index-1,0)
    #deletes proper row of label vector
    new_labels = np.delete(labels,index-1,0)
    #getting model 
    m = svm_train(new_labels,new_set,param)
    #getting weight vector,number of support vectors and norm of weight
    weight, Number_Of_Svs, w_norm = get_weight(new_set,m)
   
    return weight,Number_Of_Svs ,w_norm 

In [28]:
#deletes a data point that is not a support vector, returns weight, number of support vectors and norm of weight
def delete_dp(data,labels,m,param):
    comparison = np.arange(data.shape[0])+1
    sv_indices = m.get_sv_indices()
    
    weight=0
    Number_Of_Svs=0
    w_norm = 0

    for index in comparison:
        if index not in sv_indices:
            
            #deletes proper row of data matrix
            new_set = np.delete(data,index-1,0)
            #deletes proper row of label vector
            new_labels = np.delete(labels,index-1,0)
            #getting model 
            m = svm_train(new_labels,new_set,param)
            #getting weight vector,number of support vectors and norm of weight
            weight,Number_Of_Svs, w_norm= get_weight(new_set,m)
            break
            
    return weight, Number_Of_Svs , w_norm

In [29]:
#linear kernel wih C=10
param = '-t 0 -c 10'
m = svm_train(train_labels,train_matrix,param)
#getting weight vector,number of support vectors and norm of weight
weight, Number_Of_Svs, w_norm = get_weight(train_matrix,m)

comparison = w_norm #to use in comparison

print(f"Weight: {weight}")
print(f"Norm of the weight: {w_norm}")
print(f"Number of Support Vectors: {Number_Of_Svs}")

Weight: [-1.30726513  0.52514823  1.34794553  1.38300601  1.61374734  0.15936149
  0.13144559 -1.8510571  -0.0279832  -0.07509238  0.07069583  2.72990177
  0.44450478]
Norm of the weight: 4.4101043605317525
Number of Support Vectors: 51


### Removing one data point that is not a support vector

In [30]:
#removing one data point that is not a support vector.
weight, Number_Of_Svs, w_norm= delete_dp(train_matrix,train_labels,m,param)

print(f"Weight: {weight}")
print(f"Norm of the weight: {w_norm}")
print(f"Number of Support Vectors: {Number_Of_Svs}")

Weight: [-1.30726513  0.52514823  1.34794553  1.38300601  1.61374734  0.15936149
  0.13144559 -1.8510571  -0.0279832  -0.07509238  0.07069583  2.72990177
  0.44450478]
Norm of the weight: 4.410104360531701
Number of Support Vectors: 51


### Removing one of the support vectors

In [31]:
#removing one of the support vectors
sv_indices = m.get_sv_indices()
weight, Number_Of_Svs, w_norm = delete_sv(train_matrix,train_labels,m,param,sv_indices[1])

print(f"Weight: {weight}")
print(f"Norm of the weight: {w_norm}")
print(f"Number of Support Vectors: {Number_Of_Svs}")

Weight: [-1.35468538  0.52540875  1.34077722  1.38151342  1.63622204  0.22703252
  0.16792452 -1.97393535 -0.00791992 -0.16670834  0.01305132  2.50296982
  0.51196652]
Norm of the weight: 4.3618651742011005
Number of Support Vectors: 53


### Statistics about removing one of the support vectors

In [32]:
#number of support vectors
number = m.get_nr_sv()
#to store informations
margin_inc = 0
margin_dec = 0
margin_stb = 0

#iterates number of support vectors times
for indice in sv_indices:
    weight, Number_Of_Svs, w_norm = delete_sv(train_matrix,train_labels,m,param,indice)
    #if norm of weight decreased, margin increased
    if w_norm < comparison:
        margin_inc = margin_inc+1
    #if norm of weight increased, margin decreased
    elif w_norm > comparison:
        margin_dec = margin_dec+1
    else:
        margin_stb = margin_stb+1
        


print(f"There are {number} support vectors.")
print(f"{margin_inc} times norm of weight decreased and margin increased. Also, elements of weight vector changed.")
print(f"{margin_dec} times norm of weight increased and margin decreased. Also, elements of weight vector changed.")
print(f"{margin_stb} times norm of weight and margin did not change.")

There are 51 support vectors.
36 times norm of weight decreased and margin increased. Also, elements of weight vector changed.
15 times norm of weight increased and margin decreased. Also, elements of weight vector changed.
0 times norm of weight and margin did not change.


### Bonus Task - 10 pts

Use Python and [CVXOPT](http://cvxopt.org) QP solver to implement the hard margin SVM. 

In [33]:
# takes a matrix as 'data', its ground-truth 'labels' and trained parameters weight as 'w' and bias as 'b'
# returns accuracy in percentage form.
def get_accuracy(data,labels,w,b):
    result = np.sign(np.dot(data, w) +b).astype(int) # sign(w^Tx + bias) as an integer 1 or -1
    mul = np.multiply(result, labels) # Ground truth labels and predictions are multiplied. 
    # -1 * -1 makes 1, 1 * 1 makes 1 so this is correct classification.
    acc = np.count_nonzero(mul == 1) / len(labels) # Correct classifications rate.
    return acc*100 # % form

In [34]:
from cvxopt import matrix, solvers # import required libraries
X = np.array([[0, 0], [2, 2],[2,0],[3,0]]) # our toy data set from slides
y = np.array([-1, -1, 1, 1])

n ,dim = X.shape
# initialize variables 
Q = matrix(np.identity(dim + 1, dtype=np.float)) # (m+1)x(m+1) identity matrix (0,0) element will be made 0
p = matrix(np.zeros((dim + 1,), dtype=np.float)) # (m+1)x1 matrix of zeros
A = matrix(np.zeros((n, dim + 1), dtype=np.float)) # nx(m+1) matrix
c = -matrix(np.ones((n,), dtype=np.float)) # nx1 matrix full of -1s

Q[0, 0] = 0

for i in range(n):
    A[i, 0] = -y[i]* 1. # first column of A is negated label values
    A[i, 1:] = -y[i]*X[i, :]* 1. # other columns are  -y(i)*x(i,:)

# cvxopt solver asks constraints in less than or equal to format 
# we calculated Au>=c in the class, so we transformed equation to -Au <= -c
# Thats why we negated A and c
# solver returns a vector [b,w]
#solvers.options['show_progress'] = False # suppress output
sol = solvers.qp(Q,p,A,c)

w = np.zeros(dim,) # weight
b = sol["x"][0] # bias

for i in range(1, dim + 1):
    w[i - 1] = sol["x"][i] # get weights from solution


print(f"weight: {w}")
print(f"bias: {b}")
train_accuracy = get_accuracy(X,y,w,b) # get the accuracy of the model
print(f"Train classification accuracy: {train_accuracy}")

     pcost       dcost       gap    pres   dres
 0:  3.2653e-01  1.9592e+00  6e+00  2e+00  4e+00
 1:  1.5796e+00  8.5663e-01  7e-01  2e-16  1e-15
 2:  1.0195e+00  9.9227e-01  3e-02  4e-16  9e-16
 3:  1.0002e+00  9.9992e-01  3e-04  2e-16  2e-15
 4:  1.0000e+00  1.0000e+00  3e-06  2e-16  1e-15
 5:  1.0000e+00  1.0000e+00  3e-08  1e-16  8e-16
Optimal solution found.
weight: [ 1.00000001 -1.00000001]
bias: -1.0000000134075104
Train classification accuracy: 100.0
